# Thêm tính năng Generate Exercises từ Chunking Questions

## 1. Cấu trúc Component mới

```javascript:1_srcContentGeneration/src/frontend/src/js/components/ExerciseButton.js
import { config } from '../config.js';

export class ExerciseButton {
    constructor(scenario, questions) {
        this.scenario = scenario;    // Scenario hiện tại
        this.questions = questions;  // Các câu hỏi của scenario
        this.button = this._createButton();
    }

    _createButton() {
        const button = document.createElement('button');
        button.className = 'generate-exercise-btn';
        button.textContent = 'Generate Exercise';
        button.addEventListener('click', async (e) => {
            await this._handleClick(e);
        });
        return button;
    }

    async _handleClick(e) {
        const btn = e.target;
        btn.disabled = true;
        btn.textContent = 'Generating...';

        try {
            const exercises = await this._generateExercises();
            this._displayExercises(exercises);
        } catch (error) {
            console.error('Error generating exercises:', error);
            alert('Failed to generate exercises');
        } finally {
            btn.disabled = false;
            btn.textContent = 'Generate Exercise';
        }
    }

    async _generateExercises() {
        // Format input cho generateQuestions API
        const generateQuestionInput = `Generate 1 English question about "${this.scenario}" focusing on these key points: ${this.questions.join(', ')}`;

        const response = await fetch(`${config.production.apiUrl}/generate-questions`, {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({
                generateQuestionInput
            })
        });

        if (!response.ok) {
            throw new Error(`HTTP error! status: ${response.status}`);
        }

        const data = await response.json();
        return data.questions;
    }

    _displayExercises(exercises) {
        const container = document.getElementById(`exercises-${this.scenario.replace(/\s+/g, '-')}`);
        container.innerHTML = `
            <div class="exercises-container">
                <h6>Practice Exercise</h6>
                ${exercises.map(exercise => `
                    <div class="exercise-item">
                        <p><strong>Question:</strong> ${exercise.question}</p>
                        <p><strong>Structure:</strong> ${exercise.structure}</p>
                        <div class="phrases">
                            <p><strong>Main phrase:</strong> ${exercise['main phrase']}</p>
                            <p><strong>Option 1:</strong> ${exercise['optional phrase 1']}</p>
                            <p><strong>Option 2:</strong> ${exercise['optional phrase 2']}</p>
                        </div>
                        <div class="translations">
                            <p><em>Question (VN):</em> ${exercise['question-vi']}</p>
                            <p><em>Structure (VN):</em> ${exercise['structure-vi']}</p>
                        </div>
                    </div>
                `).join('')}
            </div>
        `;
    }

    render() {
        return this.button;
    }
}
```

## 2. Sửa ChunkingButton để thêm Exercise Button

```javascript:1_srcContentGeneration/src/frontend/src/js/components/ChunkingButton.js
import { ExerciseButton } from './ExerciseButton.js';

export class ChunkingButton {
    // ... existing code ...

    _displayQuestions(questions) {
        const container = document.getElementById(`chunking-questions-week-${this.week}`);
        container.innerHTML = `
            <div class="questions-container">
                <h5>Speaking Practice Questions</h5>
                ${questions.scenarios.map(scenario => `
                    <div class="scenario-questions">
                        <div class="scenario-header">
                            <h6>${scenario.scenario}</h6>
                            <div id="exercise-btn-${scenario.scenario.replace(/\s+/g, '-')}"></div>
                        </div>
                        <ul>
                            ${scenario.questions.map(q => `
                                <li>${q}</li>
                            `).join('')}
                        </ul>
                        <div id="exercises-${scenario.scenario.replace(/\s+/g, '-')}" class="exercises-section"></div>
                    </div>
                `).join('')}
            </div>
        `;

        // Add exercise buttons
        questions.scenarios.forEach(scenario => {
            const btnContainer = document.getElementById(`exercise-btn-${scenario.scenario.replace(/\s+/g, '-')}`);
            const exerciseButton = new ExerciseButton(scenario.scenario, scenario.questions);
            btnContainer.appendChild(exerciseButton.render());
        });
    }
}
```

## 3. Thêm CSS mới

```css:1_srcContentGeneration/src/frontend/src/css/exercise-button.css
.generate-exercise-btn {
    padding: 0.3rem 0.8rem;
    background-color: #2196F3;
    color: white;
    border: none;
    border-radius: 4px;
    font-size: 0.9rem;
    cursor: pointer;
    margin-left: 1rem;
}

.scenario-header {
    display: flex;
    align-items: center;
    margin-bottom: 1rem;
}

.exercises-section {
    margin-top: 1rem;
    padding: 1rem;
    background-color: #f8f9fa;
    border-radius: 4px;
}

.exercise-item {
    border-left: 3px solid #2196F3;
    padding-left: 1rem;
    margin-bottom: 1rem;
}

.phrases, .translations {
    margin-top: 0.5rem;
    padding: 0.5rem;
    background-color: white;
    border-radius: 4px;
}

.translations {
    color: #666;
}
```

## 4. Flow hoạt động

1. Khi user click "Generate Chunking":
   - Tạo ra 20 câu hỏi chia cho 5 scenarios
   - Mỗi scenario hiển thị 4 câu hỏi
   - Mỗi scenario có 1 nút "Generate Exercise"

2. Khi user click "Generate Exercise":
   - Lấy scenario và questions hiện tại
   - Gọi API `/generate-questions` với input được format
   - Hiển thị exercise được tạo ra ngay dưới scenario đó

## 5. Format Data

### Input cho Generate Questions API:
```javascript
{
    "generateQuestionInput": "Generate 1 English question about \"Giới thiệu dự án hiện tại\" focusing on these key points: What is the project scope?, How long will it take?, Who are the stakeholders?, What are the main objectives?"
}
```

### Output từ API:
```javascript
{
    "questions": [
        {
            "question": "Can you explain the main objectives of your current project?",
            "structure": "The main objectives of our project are ____.",
            "main phrase": "to improve efficiency",
            // ... other fields
        }
    ]
}
```

Bạn cần tôi giải thích thêm phần nào không?

